# Logo Detection - Training Notebook

**Author:** hoangedu773  
**GitHub:** https://github.com/hoangedu773  
**Dataset:** FlickrLogos32

---

## 1. Setup Environment

In [1]:
# Cài đặt thư viện
!pip install -q ultralytics gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.5 MB/s eta 0:00:00


In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

## 2. Load Dataset từ Drive

In [ ]:
import shutil
import os

# ⚠️ SỬA ĐƯỜNG DẪN NÀY CHO ĐÚNG!
DRIVE_DATASET_PATH = '/content/drive/MyDrive/FlickrLogos32.v1i.yolov8'
COLAB_DATASET_PATH = '/content/dataset'

# Copy dataset vào Colab
if os.path.exists(DRIVE_DATASET_PATH):
    print(f"📂 Đang copy dataset từ {DRIVE_DATASET_PATH}...")
    shutil.copytree(DRIVE_DATASET_PATH, COLAB_DATASET_PATH, dirs_exist_ok=True)
    print("✅ Copy xong!")
else:
    print(f"❌ Không tìm thấy {DRIVE_DATASET_PATH}")
    print("Hãy kiểm tra lại đường dẫn!")

## 3. Chuẩn bị data.yaml

In [ ]:
# Tạo file data.yaml với đường dẫn đúng
yaml_content = f"""
path: {COLAB_DATASET_PATH}
train: train/images
val: valid/images
test: test/images

nc: 32
names: ['HP', 'adidas', 'aldi', 'apple', 'becks', 'bmw', 'carlsberg', 'chimay', 
        'cocacola', 'corona', 'dhl', 'erdinger', 'esso', 'fedex', 'ferrari', 
        'ford', 'fosters', 'google', 'guiness', 'heineken', 'milka', 'nvidia', 
        'paulaner', 'pepsi', 'rittersport', 'shell', 'singha', 'starbucks', 
        'stellaartois', 'texaco', 'tsingtao', 'ups']
"""

with open(f'{COLAB_DATASET_PATH}/data.yaml', 'w') as f:
    f.write(yaml_content)

print("✅ Đã tạo data.yaml")
!cat {COLAB_DATASET_PATH}/data.yaml

## 4. Train YOLO Model

In [ ]:
from ultralytics import YOLO

# Load YOLOv8 pretrained
model = YOLO('yolov8s.pt')  # s = small, có thể dùng n/m/l/x

# Train
results = model.train(
    data=f'{COLAB_DATASET_PATH}/data.yaml',
    epochs=50,          # Số epochs (có thể tăng lên 100)
    imgsz=640,          # Kích thước ảnh
    batch=16,           # Batch size (giảm nếu hết RAM)
    name='logo_train',  # Tên experiment
    patience=10,        # Early stopping
    save=True,
    device=0            # GPU
)

print("\n✅ TRAIN XONG!")

## 5. Lưu Model về Drive

In [ ]:
import shutil

# Đường dẫn model sau train
trained_model = '/content/runs/detect/logo_train/weights/best.pt'

# Lưu vào Drive
drive_model_path = '/content/drive/MyDrive/best_logo_model.pt'
shutil.copy(trained_model, drive_model_path)

print(f"✅ Đã lưu model vào {drive_model_path}")
print("💾 File này không mất khi session Colab tắt!")

## 6. Validate Model

In [ ]:
# Đánh giá model trên test set
val_model = YOLO(trained_model)
metrics = val_model.val(data=f'{COLAB_DATASET_PATH}/data.yaml')

print("\n📊 KẾT QUẢ:")
print(f"mAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")

## 7. Test với ảnh

In [ ]:
import cv2
from IPython.display import Image, display

# Upload ảnh test
from google.colab import files
uploaded = files.upload()

# Predict
for filename in uploaded.keys():
    results = val_model.predict(source=filename, conf=0.5, save=True)
    print(f"✅ Kết quả lưu tại: {results[0].save_dir}")
    
    # Hiển thị
    output_path = f"{results[0].save_dir}/{filename}"
    display(Image(output_path))

---

## 📝 Ghi chú

- Model được lưu tại: `/content/drive/MyDrive/best_logo_model.pt`
- Copy file này về local và đặt vào `models/best.pt`
- Sau đó chạy `app.py` để test với Gradio UI
